In [18]:
from agents import Agent,trace,Runner,function_tool, WebSearchTool
from langchain.tools import DuckDuckGoSearchResults
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown

from dotenv import load_dotenv,find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

True

In [2]:
agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")

In [3]:
with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

Why did the autonomous AI agent get kicked out of the party?

Because it kept trying to optimize everyone's conversations!


## First Agent : Search Agent

In [22]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [24]:
message = "What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))


As of July 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated AI applications:

- **LangChain**: A modular Python framework that enables developers to build applications powered by large language models (LLMs). It supports various LLMs, including OpenAI, Anthropic, and Hugging Face, and offers tools for chaining prompts, managing memory, and integrating external tools. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen is an open-source framework that facilitates the creation of multi-agent conversations using LLMs. It allows for the orchestration of autonomous or human-in-the-loop AI agents to collaborate on complex tasks. ([lollypop.design](https://lollypop.design/blog/2025/june/ai-agent-frameworks/?utm_source=openai))

- **CrewAI**: An open-source framework designed for building autonomous multi-agent systems, CrewAI assigns specialized roles to each agent, enabling collaboration to complete complex tasks. It is particularly useful in applications like research, content generation, and automation. ([lollypop.design](https://lollypop.design/blog/2025/june/ai-agent-frameworks/?utm_source=openai))

- **OpenAI Agents SDK**: Released in March 2025, this lightweight Python framework focuses on creating multi-agent workflows with comprehensive tracing and guardrails, facilitating the development of complex AI systems. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **LangGraph**: An extension of LangChain, LangGraph allows developers to define agents as state machines, providing fine-grained control over agent state and supporting complex workflows with deterministic control, retries, and branching logic. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

These frameworks are widely adopted for their flexibility, scalability, and support for complex AI agent development, catering to a range of applications from enterprise solutions to research and content generation. 